# pythonからC++へのモデルの出力

https://tatsuya-note.com/trained-model-created-in-pytorch-in-cpp/

In [2]:
import torch       # ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn  # 「ニューラルネットワーク」モジュールの別名定義

class NET(nn.Module):
    nfeatures = 288 # TODO :: hard code 4*12*6=288 # len(train_X_ch[0][0])
    print(" nfeatures :: ", nfeatures )

    M = 20 
    Mb= 6
            
    #Embedding Net 
    nfeatures_enet = int(nfeatures/4) # 72
    print(nfeatures_enet)
    # 定数（モデル定義時に必要となるもの）
    INPUT_FEATURES_enet = nfeatures_enet      # 入力（特徴）の数： 記述子の数
    LAYER1_NEURONS_enet = 50             # ニューロンの数
    LAYER2_NEURONS_enet = 50             # ニューロンの数
    OUTPUT_RESULTS_enet = M*nfeatures_enet    # 出力結果の数： 

    #Fitting Net 
    nfeatures_fnet = int(M*Mb) 
    print(nfeatures_fnet)
    # 定数（モデル定義時に必要となるもの）
    INPUT_FEATURES_fnet = nfeatures_fnet    # 入力（特徴）の数： 記述子の数
    LAYER1_NEURONS_fnet = 50     # ニューロンの数
    LAYER2_NEURONS_fnet = 50     # ニューロンの数
    OUTPUT_RESULTS_fnet = M      # 出力結果の数：

    def __init__(self):
        super().__init__()

        ##### Embedding Net #####
        # 隠れ層：1つ目のレイヤー（layer）
        self.Enet_layer1 = nn.Linear(
            self.INPUT_FEATURES_enet,                # 入力ユニット数（＝入力層）
            self.LAYER1_NEURONS_enet)                # 次のレイヤーの出力ユニット数

        # 隠れ層：2つ目のレイヤー（layer）
        self.Enet_layer2 = nn.Linear(
            self.LAYER1_NEURONS_enet,                # 入力ユニット数
            self.LAYER2_NEURONS_enet)                # 次のレイヤーの出力ユニット数
        
        # 出力層
        self.Enet_layer_out = nn.Linear(
            self.LAYER2_NEURONS_enet,                # 入力ユニット数
            self.OUTPUT_RESULTS_enet)                # 出力結果への出力ユニット数
        
        ##### Fitting net #####
        # 隠れ層：1つ目のレイヤー（layer）
        self.Fnet_layer1 = nn.Linear(
            self.INPUT_FEATURES_fnet,                # 入力ユニット数（＝入力層）
            self.LAYER1_NEURONS_fnet)                # 次のレイヤーの出力ユニット数
        
        # 隠れ層：2つ目のレイヤー（layer）
        self.Fnet_layer2 = nn.Linear(
            self.LAYER1_NEURONS_fnet,                # 入力ユニット数
            self.LAYER2_NEURONS_fnet)                # 次のレイヤーの出力ユニット数
        
        # 出力層
        self.Fnet_layer_out = nn.Linear(
            self.LAYER2_NEURONS_fnet,                # 入力ユニット数
            self.OUTPUT_RESULTS_fnet)                # 出力結果への出力ユニット数
        
    def forward(self, x):

        #Si(1/Rをカットオフ関数で処理した値）のみを抽出する
        Q1 = x[:,::4]
        NB = Q1.size()[0]
        N  = Q1.size()[1]
        # Embedding Netに代入する 
        embedded_x = nn.functional.leaky_relu(self.Enet_layer1(Q1))  
        embedded_x = nn.functional.leaky_relu(self.Enet_layer2(embedded_x)) 
        embedded_x = self.Enet_layer_out(embedded_x)  # ※最終層は線形 
        #embedded_xを(ミニバッチデータ数)xMxN (N=MaxAt*原子種数)に変換
        embedded_x = torch.reshape(embedded_x,(NB,self.M,N ))
        #入力データをNB x N x 4 の行列に変形  
        matQ = torch.reshape(x,(NB,N,4))
        #Enetの出力との掛け算
        matT = torch.matmul(embedded_x, matQ)
        # matTの次元はNB x M x 4 となっている 
        #matSを作る(ハイパーパラメータMbで切り詰める)
        matS = matT[:,:self.Mb,:]
        #matSの転置行列を作る　→　NB x 4 x Mb となる 
        matSt = torch.transpose(matS, 1, 2)
        #matDを作る( matTとmatStの掛け算) →　NB x M x Mb となる 
        matD = torch.matmul(matT, matSt)
        #matDを１次元化する。matD全体をニューラルネットに入力したいので、ベクトル化する。 
        matD1 = torch.reshape(matD,(NB,self.M*self.Mb))
        # fitting Net に代入する 
        fitD = nn.functional.leaky_relu(self.Fnet_layer1(matD1))
        fitD = nn.functional.leaky_relu(self.Fnet_layer2(fitD)) 
        fitD = self.Fnet_layer_out(fitD)  # ※最終層は線形 
        # fitDの次元はNB x M となる。これをNB x 1 x Mの行列にする
        fitD3 = torch.reshape(fitD,(NB,1,self.M))
        # fttD3とmatTの掛け算 
        matW = torch.matmul(fitD3, matT) 
        # matWはNb x 1 x  4 になっている。これをNB x 4 の2次元にする
        matW2 = torch.reshape(matW,(NB,4))
        # はじめの要素はいらないので、切り詰めてx,y,z にする
        outW = matW2[:,1:]
        
        return outW



 nfeatures ::  288
72
120


In [3]:

# # モデル（NeuralNetworkクラス）のインスタンス化
model_ring = NET()
model_ch = NET()
model_co = NET()
model_cc = NET()
model_oh = NET()
model_o = NET()
model_coc = NET()
model_coh = NET()
# <<<<<<<  if文ここまで <<<<<<<<
try:
    from torchinfo import summary
    summary(model=model_ring)
except ImportError:
    print("WARNING :: torchinfo is not installed. skip printing model summary.")
    print("WARNING :: This has no effect on the calculation.")

# 
# * モデルをロードする場合はこれを利用する

model_dir="/Volumes/portableSSD2T/14_pg/pg_cpmd/202306014_model_rotate/" # PG用
model_dir="./20230629_model_rotate/" # PG2量体用
model_dir="/Volumes/portableSSD2T/14_pg/pg_gromacs_bulk3_500K/20230721_model_rotate/" # PG, 500K
model_dir="/Volumes/portableSSD2T/14_pg/ppg725/bulk/20230719_model_rotate/" #PPG725, 300K
model_dir="/Volumes/portableSSD2T/14_pg/pg2_new_bulk/20230719_model_rotate_COC/" #PG2のCOC,COHモデル(これは失敗モデル？)
model_dir="/Volumes/portableSSD2T/14_pg/pg2_new_bulk/20230720_model_rotate_COC/" #PG2のCOC,COHモデル


print(" ------------------- ")
print(" loading ML variables from modeldir ... ")
print("")
# model_dir="model_train40percent/"
# model_ring.load_state_dict(torch.load('model_ring_weight.pth'))

#GPUが使用可能か確認
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device :: check if use GPU :: {}".format(device))
import os
import torch.multiprocessing as mp
if os.path.isfile(model_dir+'model_ch_weight.pth'):
    model_ch.load_state_dict(torch.load(model_dir+'model_ch_weight.pth'))
    model_ch_2 = model_ch.to(device) # 一旦モデルをcpuへ
    print("model_ch_2 :: {}".format(model_ch_2))
    model_ch_2.share_memory() #https://knto-h.hatenablog.com/entry/2018/05/22/130745
else:
    print("model_ch_2 is not loaded")
if os.path.isfile(model_dir+'model_co_weight.pth'):
    model_co.load_state_dict(torch.load(model_dir+'model_co_weight.pth'))
    model_co_2 = model_co.to(device)
    print("model_co_2 :: {}".format(model_co_2))
else:
    print("model_co_2 is not loaded")
if os.path.isfile(model_dir+'model_oh_weight.pth'):
    model_oh.load_state_dict(torch.load(model_dir+'model_oh_weight.pth'))
    model_oh_2 = model_oh.to(device)
    print("model_oh_2 :: {}".format(model_oh_2))
else:
    print("model_oh_2 is not loaded")
if os.path.isfile(model_dir+'model_cc_weight.pth'):
    model_cc.load_state_dict(torch.load(model_dir+'model_cc_weight.pth'))
    model_cc_2 = model_cc.to(device)
    print("model_cc_2 :: {}".format(model_cc_2))
else:
    model_cc_2 = None
    print("model_cc_2 is not loaded")
if os.path.isfile(model_dir+'model_o_weight.pth'):
    model_o.load_state_dict(torch.load(model_dir+'model_o_weight.pth'))
    model_o_2  = model_o.to(device)
    print("model_o_2 :: {}".format(model_o_2))
else:
    print("model_o_2 is not loaded")

# 以下COC/COHモデル用
if os.path.isfile(model_dir+'model_coc_weight4.pth'):
    model_coc.load_state_dict(torch.load(model_dir+'model_coc_weight4.pth'))
    model_coc_2  = model_coc.to(device)
    print("model_coc_2 :: {}".format(model_coc_2))
else:
    print("model_coc_2 is not loaded")
if os.path.isfile(model_dir+'model_coh_weight4.pth'):
    model_coh.load_state_dict(torch.load(model_dir+'model_coh_weight4.pth'))
    model_coh_2  = model_coh.to(device)
    print("model_coh_2 :: {}".format(model_coh_2))
else:
    print("model_coh_2 is not loaded")


 ------------------- 
 loading ML variables from modeldir ... 

device :: check if use GPU :: cpu
model_ch_2 is not loaded
model_co_2 is not loaded
model_oh_2 is not loaded
model_cc_2 is not loaded
model_o_2 is not loaded
model_coc_2 :: NET(
  (Enet_layer1): Linear(in_features=72, out_features=50, bias=True)
  (Enet_layer2): Linear(in_features=50, out_features=50, bias=True)
  (Enet_layer_out): Linear(in_features=50, out_features=1440, bias=True)
  (Fnet_layer1): Linear(in_features=120, out_features=50, bias=True)
  (Fnet_layer2): Linear(in_features=50, out_features=50, bias=True)
  (Fnet_layer_out): Linear(in_features=50, out_features=20, bias=True)
)
model_coh_2 :: NET(
  (Enet_layer1): Linear(in_features=72, out_features=50, bias=True)
  (Enet_layer2): Linear(in_features=50, out_features=50, bias=True)
  (Enet_layer_out): Linear(in_features=50, out_features=1440, bias=True)
  (Fnet_layer1): Linear(in_features=120, out_features=50, bias=True)
  (Fnet_layer2): Linear(in_features=50, o

In [6]:
#
# * 学習時の形を決める必要がある．pythonの場合はvector対応できたがC++の場合にはそれができない点に注意が必要だ．．．
import os

# PG用
if not os.path.isdir("202306014_model_rotate/"):
    os.mkdir("202306014_model_rotate/")

# PG2量体用
if not os.path.isdir("20230629_model_rotate/"):
    os.mkdir("20230629_model_rotate/")

# PG500K用
if not os.path.isdir("20230721_PG500K_model_rotate/"):
    os.mkdir("20230721_PG500K_model_rotate/")

# PPG725用
if not os.path.isdir("20230719_PPG725_model_rotate/"):
    os.mkdir("20230719_PPG725_model_rotate/")

# PG2のCOC,COH用
if not os.path.isdir("20230719_PG2_model_rotate_COC/"):
    os.mkdir("20230719_PG2_model_rotate_COC/")

# PG2のCOC,COH用
if not os.path.isdir("20230720_PG2_model_rotate_COC/"):
    os.mkdir("20230720_PG2_model_rotate_COC/")

modeldir = "20230720_PG2_model_rotate_COC/"

# 学習時の入力サンプル
example_input = torch.rand(1,288).to(device)

# 学習済みモデルのトレース
traced_net = torch.jit.trace(model_ch_2,example_input)
# 変換モデルの出力
traced_net.save(modeldir+"model_ch.pt")

# 学習済みモデルのトレース
traced_net = torch.jit.trace(model_co_2,example_input)
# 変換モデルの出力
traced_net.save(modeldir+"model_co.pt")


# 学習済みモデルのトレース
traced_net = torch.jit.trace(model_oh_2,example_input)
# 変換モデルの出力
traced_net.save(modeldir+"model_oh.pt")


# 学習済みモデルのトレース
traced_net = torch.jit.trace(model_cc_2,example_input)
# 変換モデルの出力
traced_net.save(modeldir+"model_cc.pt")

# 学習済みモデルのトレース
traced_net = torch.jit.trace(model_o_2,example_input)
# 変換モデルの出力
traced_net.save(modeldir+"model_o.pt")

# 学習済みモデルのトレース
traced_net = torch.jit.trace(model_coc_2,example_input)
# 変換モデルの出力
traced_net.save(modeldir+"model_coc.pt")

# 学習済みモデルのトレース
traced_net = torch.jit.trace(model_coh_2,example_input)
# 変換モデルの出力
traced_net.save(modeldir+"model_coh.pt")

